# Transfer Learning Example
Here is some code demonstrating how to do transfer learning on the cifar dataset using a vision transformer and a CNN (Resnet). The code takes a bit over an hour to run on a GPU in google colab so don't feel like you need to run this, but this is a fun little example of how you would apply transfer learning to some more powerful models.

In [2]:
!pip install transformers

ERROR: unknown command "transformers"



# Pretrained ViT

In [ ]:
# Import necessary libraries
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTForImageClassification, ViTImageProcessor, Trainer, TrainingArguments

def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the training and testing datasets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Initialize the feature extractor
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

# Define a custom dataset class to apply the feature extractor
class CustomCIFAR10Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, feature_extractor):
        self.dataset = dataset
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        # Ensure the image is in the correct format (e.g., PIL Image)
        if isinstance(image, torch.Tensor):
            # Convert the tensor to a PIL Image
            image = transforms.ToPILImage()(image)
        encoding = self.feature_extractor(images=image, return_tensors="pt")
        pixel_values = encoding['pixel_values'].squeeze()
        return {'pixel_values': pixel_values, 'labels': label}

num_labels = 10
id2label = {i: f"label_{i}" for i in range(num_labels)}
label2id = {f"label_{i}": i for i in range(num_labels)}
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

initial_trainable_params = count_trainable_parameters(model)
print(f"Trainable parameters before freezing: {initial_trainable_params}")

# Modify the classifier to match the number of classes in CIFAR-10
in_features = model.classifier.in_features
model.classifier = torch.nn.Linear(in_features, 10)

# Freeze all layers except the classifier
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

model.to(device)

final_trainable_params = count_trainable_parameters(model)
print(f"Trainable parameters after freezing: {final_trainable_params}")

# Create DataLoader instances
train_loader = DataLoader(CustomCIFAR10Dataset(train_dataset, feature_extractor), batch_size=32, shuffle=True)
test_loader = DataLoader(CustomCIFAR10Dataset(test_dataset, feature_extractor), batch_size=32, shuffle=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    run_name='vit-cifar-transfer-learning',
    report_to="none"
)

# Define a function to compute metrics
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {'accuracy': (preds == p.label_ids).astype(float).mean().item()}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_loader.dataset,
    eval_dataset=test_loader.dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Test Accuracy: {eval_results['eval_accuracy']:.4f}")

c:\Users\kaden\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pretrained CNN

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ResNetForImageClassification, AutoImageProcessor, Trainer, TrainingArguments

def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the training and testing datasets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Initialize the feature extractor
feature_extractor = AutoImageProcessor.from_pretrained('microsoft/resnet-50')

# Define a custom dataset class to apply the feature extractor
class CustomCIFAR10Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, feature_extractor):
        self.dataset = dataset
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        # Ensure the image is in the correct format (e.g., PIL Image)
        if isinstance(image, torch.Tensor):
            # Convert the tensor to a PIL Image
            image = transforms.ToPILImage()(image)
        encoding = self.feature_extractor(images=image, return_tensors="pt")
        pixel_values = encoding['pixel_values'].squeeze()
        return {'pixel_values': pixel_values, 'labels': label}

num_labels = 10
id2label = {i: f"label_{i}" for i in range(num_labels)}
label2id = {f"label_{i}": i for i in range(num_labels)}
model = ResNetForImageClassification.from_pretrained(
    'microsoft/resnet-50',
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

initial_trainable_params = count_trainable_parameters(model)
print(f"Trainable parameters before freezing: {initial_trainable_params}")

# Access the classifier's input features
in_features = model.classifier[1].in_features
# in_features = 65536

# Modify the classifier to match the number of classes in CIFAR-10
model.classifier = torch.nn.Sequential(
    torch.nn.Flatten(start_dim=1, end_dim=-1),
    torch.nn.Linear(in_features, 10)
)

# Freeze all layers except the classifier
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

model.to(device)

final_trainable_params = count_trainable_parameters(model)
print(f"Trainable parameters after freezing: {final_trainable_params}")

# Create DataLoader instances
train_loader = DataLoader(CustomCIFAR10Dataset(train_dataset, feature_extractor), batch_size=32, shuffle=True)
test_loader = DataLoader(CustomCIFAR10Dataset(test_dataset, feature_extractor), batch_size=32, shuffle=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    run_name='resnet-cifar-transfer-learning',
    report_to="none"
)

# Define a function to compute metrics
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {'accuracy': (preds == p.label_ids).astype(float).mean().item()}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_loader.dataset,
    eval_dataset=test_loader.dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Test Accuracy: {eval_results['eval_accuracy']:.4f}")


Files already downloaded and verified
Files already downloaded and verified


preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([10, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters before freezing: 23528522
Trainable parameters after freezing: 20490


Epoch,Training Loss,Validation Loss,Accuracy
1,1.985400,1.988141,0.516800
2,1.841800,1.839359,0.536100


# Subset data to go faster

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from transformers import ResNetForImageClassification, AutoImageProcessor, Trainer, TrainingArguments

def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the training and testing datasets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create subsets containing only the first 3200 samples
train_subset = Subset(train_dataset, indices=range(320))
test_subset = Subset(test_dataset, indices=range(64))

# Initialize the feature extractor
feature_extractor = AutoImageProcessor.from_pretrained('microsoft/resnet-50')

# Define a custom dataset class to apply the feature extractor
class CustomCIFAR10Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, feature_extractor):
        self.dataset = dataset
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        # Ensure the image is in the correct format (e.g., PIL Image)
        if isinstance(image, torch.Tensor):
            # Convert the tensor to a PIL Image
            image = transforms.ToPILImage()(image)
        encoding = self.feature_extractor(images=image, return_tensors="pt")
        pixel_values = encoding['pixel_values'].squeeze()
        return {'pixel_values': pixel_values, 'labels': label}

num_labels = 10
id2label = {i: f"label_{i}" for i in range(num_labels)}
label2id = {f"label_{i}": i for i in range(num_labels)}
model = ResNetForImageClassification.from_pretrained(
    'microsoft/resnet-50',
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

initial_trainable_params = count_trainable_parameters(model)
print(f"Trainable parameters before freezing: {initial_trainable_params}")

# Access the classifier's input features
in_features = model.classifier[1].in_features
# in_features = 65536

# Modify the classifier to match the number of classes in CIFAR-10
model.classifier = torch.nn.Sequential(
    torch.nn.Flatten(start_dim=1, end_dim=-1),
    torch.nn.Linear(in_features, 10)
)

# Freeze all layers except the classifier
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

model.to(device)

final_trainable_params = count_trainable_parameters(model)
print(f"Trainable parameters after freezing: {final_trainable_params}")

# Create DataLoader instances for the subsets
train_loader = DataLoader(CustomCIFAR10Dataset(train_subset, feature_extractor), batch_size=32, shuffle=True)
test_loader = DataLoader(CustomCIFAR10Dataset(test_subset, feature_extractor), batch_size=32, shuffle=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    run_name='resnet-cifar-transfer-learning',
    report_to="none"
)

# Define a function to compute metrics
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {'accuracy': (preds == p.label_ids).astype(float).mean().item()}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_loader.dataset,
    eval_dataset=test_loader.dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Test Accuracy: {eval_results['eval_accuracy']:.4f}")
